In [1]:
import struct
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import math
import os
from enum import IntEnum

In [2]:
def add_flight_states_and_events(fig, flight_states_df, event_info_df):
    for _, state in flight_states_df.iterrows():
        fig.add_vline(x=state['ts'], line_width=1, line_dash="dash", line_color="green", annotation_text=state['state'])
#         fig.add_trace(
#         go.Scatter(
#             x=[5, 5, 6, 6], 
#             y=[-100,  100, 100, -100], 
#             fill="toself",
#             mode='lines',
#             text= state['state'],
#             name = state['state'],
#             opacity=0
#         )
#     )
    for _, event in event_info_df.iterrows():
        if event['out_idx'] == 0:
            fig.add_vline(x=event['ts'], line_width=1, line_dash="dot", line_color="orange",
                          annotation_text=event['event'], annotation_y=0.03)


def plot_imu_data(imu_df, flight_states_df, event_info_df, ret=False):
    fig_ax = go.Figure()
    fig_ay = go.Figure()
    fig_az = go.Figure()
    fig_gx = go.Figure()
    fig_gy = go.Figure()
    fig_gz = go.Figure()
    for imu_id, one_imu in imu_df.groupby('id'):
        fig_ax.add_scatter(x=one_imu.ts, y=one_imu.Ax, name=imu_id, mode='lines')
        fig_ay.add_scatter(x=one_imu.ts, y=one_imu.Ay, name=imu_id, mode='lines')
        fig_az.add_scatter(x=one_imu.ts, y=one_imu.Az, name=imu_id, mode='lines')
        fig_gx.add_scatter(x=one_imu.ts, y=one_imu.Gx, name=imu_id, mode='lines')
        fig_gy.add_scatter(x=one_imu.ts, y=one_imu.Gy, name=imu_id, mode='lines')
        fig_gz.add_scatter(x=one_imu.ts, y=one_imu.Gz, name=imu_id, mode='lines')

    add_flight_states_and_events(fig_ax, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_ay, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_az, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_gx, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_gy, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_gz, flight_states_df, event_info_df)

    fig_ax.update_layout(
        title="IMU Acc_x",
        yaxis_title="Acc_x [g]",
        legend_title="IMU ID",
        **layout_args
    )

    fig_ay.update_layout(
        title="IMU Acc_y",
        yaxis_title="Acc_y [g]",
        legend_title="IMU ID",
        **layout_args
    )

    fig_az.update_layout(
        title="IMU Acc_z",
        yaxis_title="Acc_z [g]",
        legend_title="IMU ID",
        **layout_args
    )

    fig_gx.update_layout(
        title="IMU Gyro_x",
        yaxis_title="Gyro_x [dps]",
        legend_title="IMU ID",
        **layout_args
    )

    fig_gy.update_layout(
        title="IMU Gyro_y",
        yaxis_title="Gyro_y [dps]",
        legend_title="IMU ID",
        **layout_args
    )

    fig_gz.update_layout(
        title="IMU Gyro_z",
        yaxis_title="Gyro_z [dps]",
        legend_title="IMU ID",
        **layout_args
    )

    if ret:
        return [fig_ax, fig_ay, fig_az, fig_gx, fig_gy, fig_gz]
    else:
        fig_ax.show()
        fig_ay.show()
        fig_az.show()
        fig_gx.show()
        fig_gy.show()
        fig_gz.show()
        return None
    
def plot_magneto_data(magneto_df, flight_states_df, event_info_df, ret=False):
    fig_mx = go.Figure()
    fig_my = go.Figure()
    fig_mz = go.Figure()
    fig_mx.add_scatter(x=magneto_df.ts, y=magneto_df['mx'], name="magneto x", mode='lines')
    fig_my.add_scatter(x=magneto_df.ts, y=magneto_df['my'], name="magneto y", mode='lines')
    fig_mz.add_scatter(x=magneto_df.ts, y=magneto_df['mz'], name="magneto z", mode='lines')

    add_flight_states_and_events(fig_mx, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_my, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_mz, flight_states_df, event_info_df)


    fig_mx.update_layout(
        title="Magneto X",
        yaxis_title="Mag_x [mG]",
        legend_title="MAG ID",
        **layout_args
    )

    fig_my.update_layout(
        title="Magneto Y",
        yaxis_title="Mag_y [mG]",
        legend_title="MAG ID",
        **layout_args
    )

    fig_mz.update_layout(
        title="Magneto Z",
        yaxis_title="Mag_z [mG]",
        legend_title="MAG ID",
        **layout_args
    )

    if ret:
        return [fig_mx, fig_my, fig_mz]
    else:
        fig_mx.show()
        fig_my.show()
        fig_mz.show()
        return None


def plot_baro_data(baro_df, flight_states_df, event_info_df, ret=False):
    fig_temp = go.Figure()
    fig_press = go.Figure()
    for baro_id, one_baro in baro_df.groupby('id'):
        fig_temp.add_scatter(x=one_baro.ts, y=one_baro['T'], name=baro_id, mode='lines')
        fig_press.add_scatter(x=one_baro.ts, y=one_baro['P'], name=baro_id, mode='lines')

    add_flight_states_and_events(fig_temp, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_press, flight_states_df, event_info_df)

    deg = u'\N{DEGREE SIGN}'
    fig_temp.update_layout(
        title="BARO Temperature",
        yaxis_title=f"T [{deg}C]",
        legend_title="BARO ID",
        **layout_args
    )

    fig_press.update_layout(
        title="BARO Pressure",
        yaxis_title="P [hPa]",
        legend_title="BARO ID",
        **layout_args
    )

    if ret:
        return [fig_temp, fig_press]
    else:
        fig_temp.show()
        fig_press.show()
        return None


def plot_flight_info(flight_info_df, filtered_data_info_df, flight_states_df, event_info_df, include_bounds = True, ret=False):
    fig_height = go.Figure()
    fig_vel = go.Figure()
    fig_acc = go.Figure()

    fig_height.add_scatter(x=flight_info_df.ts, y=flight_info_df['height'], name="height", mode='lines')
    fig_height.add_scatter(x=filtered_data_info_df.ts, y=filtered_data_info_df['altitude_agl'], name="altitude_agl", mode='lines')
    fig_height.add_scatter(x=filtered_data_info_df.ts, y=filtered_data_info_df['filtered_altitude_AGL'], name='filtered_altitude_agl', mode='lines')
        
    if include_bounds:
        fig_height.add_scatter(x=flight_info_df.ts, y=flight_info_df['lower_bound'], name="lower_bound", mode='lines')
        fig_height.add_scatter(x=flight_info_df.ts, y=flight_info_df['upper_bound'], name="upper_bound", mode='lines')

    fig_vel.add_scatter(x=flight_info_df.ts, y=flight_info_df['velocity'], name="velocity", mode='lines')

    fig_acc.add_scatter(x=flight_info_df.ts, y=flight_info_df['acceleration'], name="acceleration", mode='lines')
    fig_acc.add_scatter(x=filtered_data_info_df.ts, y=filtered_data_info_df['raw_acceleration'], name="raw_acceleration", mode='lines')
    fig_acc.add_scatter(x=filtered_data_info_df.ts, y=filtered_data_info_df['filtered_acceleration'], name="filtered_acceleration", mode='lines')

    add_flight_states_and_events(fig_height, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_vel, flight_states_df, event_info_df)
    add_flight_states_and_events(fig_acc, flight_states_df, event_info_df)

    fig_height.update_layout(
        title="Height",
        yaxis_title="Height [m]",
        **layout_args
    )

    fig_vel.update_layout(
        title="Velocity",
        yaxis_title="Velocity [m/s]",
        **layout_args
    )

    squared = u'\N{SUPERSCRIPT TWO}'
    fig_acc.update_layout(
        title="Acceleration",
        yaxis_title=f"Acceleration [m/s{squared}]",
        **layout_args
    )
    if ret:
        return [fig_height, fig_vel, fig_acc]
    else:
        fig_height.show()
        fig_vel.show()
        fig_acc.show()
        return None


def figures_to_html(imu_plots, baro_plots, magneto_plots, state_plots, filename):
    '''Saves a list of plotly figures in an html file.'''

    dashboard = open(filename, 'w')
    head = """<html><head>
    <style>
        body {background-color: black;}
        div  {display: inline-block;margin: 2px; padding:2px;}
        h1 {font-size: 3em; color: floralwhite; font-family: 'Courier New', monospace}
        h2 {font-size: 2em; color: floralwhite; font-family: 'Courier New', monospace; margin-bottom: 2px}
        .container { white-space: nowrap; }
    </style>
    </head><body><center>\n"""
    dashboard.write(head)

    #         span {
    #             float: left;
    #             white-space:nowrap;
    #         }

    dashboard.write('<h1>IMU Data - Acceleration</h1>')
    #dashboard.write('<h1>IMU Data - Acceleration</h1><div class="container">')

    add_js = True
    xyz = 'XYZ'
    i = 0
    for fig in imu_plots[:3]:
        inner_html = pyo.plot(
            fig, include_plotlyjs=add_js, output_type='div'
        )
        dashboard.write(f'<h2>Acc {xyz[i]}</h2><div class="container">')
        dashboard.write(inner_html)
        dashboard.write('</div>')
        add_js = False
        i += 1

    dashboard.write('<br/>')

    i = 0
    dashboard.write('<h1>IMU Data - Gyros</h1>')
    for fig in imu_plots[3:]:
        inner_html = pyo.plot(
            fig, include_plotlyjs=add_js, output_type='div'
        )
        dashboard.write(f'<h2>Gyro {xyz[i]}</h2><div class="container">')
        dashboard.write(inner_html)
        dashboard.write('</div>')
        add_js = False
        i += 1

    dashboard.write('<br/>')

    dashboard.write('<h1>Barometer Data</h1>')
    xyz = ['Temperature', 'Pressure']
    i = 0
    for fig in baro_plots:
        inner_html = pyo.plot(
            fig, include_plotlyjs=add_js, output_type='div'
        )
        dashboard.write(f'<h2>{xyz[i]}</h2><div class="container">')
        dashboard.write(inner_html)
        dashboard.write('</div>')
        add_js = False
        i += 1

    dashboard.write('<br/>')
    
    dashboard.write('<h1>Magnetometer Data</h1>')
    xyz = 'XYZ'
    i = 0
    for fig in magneto_plots:
        inner_html = pyo.plot(
            fig, include_plotlyjs=add_js, output_type='div'
        )
        dashboard.write(f'<h2>Magneto {xyz[i]}</h2><div class="container">')
        dashboard.write(inner_html)
        dashboard.write('</div>')
        add_js = False
        i += 1

    dashboard.write('<br/>')

    dashboard.write('<h1>State Estimation Data</h1>')

    xyz = ['AGL Altitude & Height', 'Velocity', 'Acceleration']
    i = 0
    for fig in state_plots:
        inner_html = pyo.plot(
            fig, include_plotlyjs=add_js, output_type='div'
        )
        dashboard.write(f'<h2>{xyz[i]}</h2><div class="container">')
        dashboard.write(inner_html)
        dashboard.write('</div>')
        add_js = False
        i += 1

    dashboard.write('<br/>')

    dashboard.write("</center></body></html>" + "\n")

In [3]:
FLIGHT_MAP = {
    0: 'INVALID',
    1: 'MOVING',
    2: 'READY',
    3: 'THRUSTING_1',
    4: 'THRUSTING_2',
    5: 'COASTING',
    6: 'TRANSONIC_1',
    7: 'SUPERSONIC',
    8: 'TRANSONIC_2',
    9: 'APOGEE',
    10: 'DROGUE',
    11: 'MAIN',
    12: 'TOUCHDOWN',
    0x7FFFFFFF: 'HEHE2'
}

DROP_MAP = {
    0: 'INVALID',
    1: 'MOVING',
    2: 'READY',
    3: 'DROGUE CHUTE',
    4: 'MAIN CHUTE',
    0x7FFFFFFF: 'HEHE'
}

EVENT_MAP = {
    0: 'EV_MOVING',
    1: 'EV_READY',
    2: 'EV_LIFTOFF',
    3: 'EV_MAX_V',
    4: 'EV_APOGEE',
    5: 'EV_POST_APOGEE',
    6: 'EV_TOUCHDOWN',
    7: 'EV_CUSTOM_1',
    8: 'EV_CUSTOM_2',
    9: 'EV_MACHTIMER',
    0xFFFFFFFF: 'EV_HEHE'
}

ERROR_MAP = {
    0x01 : 'ERR_NO_CONFIG',
    0x02 : 'ERR_NO_PYRO',
    0x04 : 'ERR_LOG_FULL',
    0x08 : 'ERR_USB_CONNECTED',
    0x10 : 'ERR_BAT_LOW',
    0x20 : 'ERR_BAT_CRITICAL',
    0x40 : 'ERR_IMU_0',
    0x80 : 'ERR_IMU_1',
    0x100 : 'ERR_IMU_2',
    0x200 : 'ERR_BARO_0',
    0x400 : 'ERR_BARO_1',
    0x800 : 'ERR_BARO_2',
    0x1000 : 'ERR_FILTER',
    0x2000 : 'ERR_HARD_FAULT',
    0xFFFFFFFF: 'ERR_HEHE'
}

layout_font = dict(family="Courier New, monospace", size=12)

legend = dict(orientation='h', yanchor="bottom", y=1.02, xanchor="right", x=1)

layout_args = dict(xaxis_title="Timestamp [s]",
                   template='plotly_dark',
                   legend=legend,
                   font=layout_font,
                   width=1500, height=800)

class REC_TYPE(IntEnum):
  IMU                = 1 << 4   # 0x20
  BARO               = 1 << 5   # 0x40
  MAGNETO            = 1 << 6   # 0x80
  ACCELEROMETER      = 1 << 7   # 0x100
  FLIGHT_INFO        = 1 << 8   # 0x200
  ORIENTATION_INFO   = 1 << 9   # 0x400
  FILTERED_DATA_INFO = 1 << 10  # 0x800
  FLIGHT_STATE       = 1 << 11  # 0x1000
  COVARIANCE_INFO    = 1 << 12  # 0x2000
  SENSOR_INFO        = 1 << 13  # 0x4000
  EVENT_INFO         = 1 << 14  # 0x8000
  ERROR_INFO         = 1 << 15  # 0x10000

REC_ID_MASK = 0x0000000F

def get_id_from_record_type(rec_type):
    return rec_type & REC_ID_MASK

def get_record_type_without_id(rec_type):
    return rec_type & ~REC_ID_MASK

In [7]:
log = 'flight_logs/sensor_id/teraterm3.log'
STATE_MAP = FLIGHT_MAP

base_name = log.split('/')[-1].split('.')[0]

plot_output_dir = f'output/{base_name}'
raw_output_dir = f'output/{base_name}/raw'
processed_output_dir = f'output/{base_name}/processed'

for out_dir in [plot_output_dir, raw_output_dir, processed_output_dir]:
    if not os.path.exists(out_dir):
            os.makedirs(out_dir)

In [8]:
with open(log, 'r') as f:
    log_str = f.read()

log_str_formatted = log_str.replace('\r\n', ' ')
log_str_formatted = log_str_formatted.replace('\n', ' ')
log_str_formatted = log_str_formatted.replace('  ', ' ')
log_str_formatted = log_str_formatted.strip()

In [9]:
log_b = bytes.fromhex(log_str_formatted)

print(log_b[:23])

b'!\x00\x00\x00\xb5N\x00\x00Hz\x01\x00\xe2\t\x00\x00"\x00\x00\x00\xb5N\x00'


In [10]:
imu = []
baro = []
accelerometer = []
flight_info = []
orientation_info = []
filtered_data_info = []
flight_states = []
covariance_info = []
sensor_info = []
event_info = []
error_info = []
magneto = []
i = 0
first_ts = -1
last_ts = -1
try:
    while i < len(log_b):
        t = struct.unpack('<L', log_b[i:i + 4])[0]
        sensor_id = get_id_from_record_type(t)
        t_without_id = get_record_type_without_id(t)
        i += 4
        if t_without_id == REC_TYPE.IMU:
            ts, gyro_x, gyro_y, gyro_z, acc_x, acc_y, acc_z = struct.unpack('<Lhhhhhh', log_b[i: i + 16])
            if first_ts == -1:
                first_ts = ts
            #print(f"{ts} IMU {t-1}: Gx: {gyro_x}, Gy: {gyro_y}, Gz: {gyro_z}, Ax: {acc_x}, Ay: {acc_y}, Az: {acc_z}")
            imu.append({'ts': ts,
                        'id': f'IMU{sensor_id}',
                        'Gx': gyro_x,
                        'Gy': gyro_y,
                        'Gz': gyro_z,
                        'Ax': acc_x,
                        'Ay': acc_y,
                        'Az': acc_z})
            #print(imu_data)
            i += 4 + 6 + 6  #ts + 6 x int16
        elif t_without_id ==  REC_TYPE.BARO:
            ts, pressure, temperature = struct.unpack('<LLL', log_b[i: i + 12])
            if first_ts == -1:
                first_ts = ts
            #print(f"{ts} BARO {t-4}: P: {pressure}, T: {temperature}")
            baro.append({'ts': ts,
                         'id': f'BARO{sensor_id}',
                         'T': temperature,
                         'P': pressure})
            #print(baro_data)
            i += 4 + 4 + 4
        elif t_without_id ==  REC_TYPE.MAGNETO:
            ts, mag_x, mag_y, mag_z = struct.unpack('<Lfff', log_b[i: i + 16])
            if first_ts == -1:
                first_ts = ts
            magneto.append({'ts': ts,
                            'mx': mag_x,
                            'my': mag_y,
                            'mz': mag_z,})
            i += 4 + 4 + 4 + 4
        elif t_without_id == REC_TYPE.ACCELEROMETER:
            ts, acc_x, acc_y, acc_z = struct.unpack('<Lbbb', log_b[i: i + 7])
            if first_ts == -1:
                first_ts = ts
            accelerometer.append({'ts': ts,
                            'acc_x': acc_x,
                            'acc_y': acc_y,
                            'acc_z': acc_z,})
            i += 4 + 1 + 1 + 1 + 1
        elif t_without_id == REC_TYPE.FLIGHT_INFO:
            ts, height, velocity, acceleration = struct.unpack('<Lfff', log_b[i: i + 16])
            if first_ts == -1:
                first_ts = ts
            #print(f"{ts} FLIGHT_INFO: Height: {height}, Velocity: {velocity}, Acc: {acceleration}")
            flight_info.append({'ts': ts,
                                'height': height,
                                'velocity': velocity,
                                'acceleration': acceleration})
            #print(flight_info_data)
            i += 4 + 4 + 4 + 4
        elif t_without_id == REC_TYPE.ORIENTATION_INFO:
            ts, est_0, est_1, est_2, est_3, raw_0, raw_1, raw_2, raw_3 = struct.unpack('<Lhhhhhhhh', log_b[i: i + 20])
            if first_ts == -1:
                first_ts = ts
            orientation_info.append({'ts': ts,
                                       'q0_estimated': est_0,
                                       'q1_estimated': est_1,
                                       'q2_estimated': est_2,
                                       'q3_estimated': est_3,
                                       'q0_raw': raw_0,
                                       'q1_raw': raw_1,
                                       'q2_raw': raw_2,
                                       'q3_raw': raw_3})
            #print(orientation_info)
            i += 4 + 4 + 4 + 4 + 4
        elif t_without_id == REC_TYPE.FILTERED_DATA_INFO:
            ts, altitude_agl, raw_acceleration, filtered_altitude_AGL, filtered_acceleration = struct.unpack('<Lffff', log_b[i: i + 20])
            if first_ts == -1:
                first_ts = ts
            #print(f"{ts} FLIGHT_INFO: Height: {height}, Velocity: {velocity}, Acc: {acceleration}")
            filtered_data_info.append({'ts': ts,
                                       'altitude_agl': altitude_agl,
                                       'raw_acceleration': raw_acceleration,
                                       'filtered_altitude_AGL': filtered_altitude_AGL,
                                       'filtered_acceleration': filtered_acceleration})
            #print(flight_info_data)
            i += 4 + 4 + 4 + 4 + 4
        elif t_without_id == REC_TYPE.FLIGHT_STATE:
            #print("FLIGHT_STATE")
            ts, state = struct.unpack('<LL', log_b[i: i + 8])
            if first_ts == -1:
                first_ts = ts
            flight_states.append({'ts': ts, 'state': state})
            #print(f"{ts} FLIGHT STATE: State: {state}")
            i += 4 + 4
        elif t_without_id == REC_TYPE.COVARIANCE_INFO:
            ts, height_cov, velocity_cov = struct.unpack('<Lff', log_b[i: i + 12])
            if first_ts == -1:
                first_ts = ts
            #print(f"{ts} COVARIANCE_INFO: Height cov: {height_cov}, Velocity cov: {velocity_cov}")
            covariance_info.append({'ts': ts,
                                    'height_cov': height_cov,
                                    'velocity_cov': velocity_cov})
            #print(flight_info_data)
            i += 4 + 4 + 4
        elif t_without_id == REC_TYPE.SENSOR_INFO:
            #print(f"Sensor info found at {i}")
            ts, imu_0, imu_1, imu_2, baro_0, baro_1, baro_2 = struct.unpack('<LBBBBBB', log_b[i: i + 10])
            if first_ts == -1:
                first_ts = ts
            sensor_info.append({'ts': ts,
                                'imu_0': imu_0,
                                'imu_1': imu_1,
                                'imu_2': imu_2,
                                'baro_0': baro_0, 'baro_1': baro_1, 'baro_2': baro_2})
            i += 4 + 6 + 2  # +2 is because of the padding
        elif t_without_id == REC_TYPE.EVENT_INFO:
            #print(f"Event info found at {i}")
            ts, event, out_idx = struct.unpack('<LLB', log_b[i: i + 9])
            if first_ts == -1:
                first_ts = ts
            event_info.append({'ts': ts,
                               'event': event,
                               'out_idx': out_idx})
            i += 4 + 4 + 1 + 3  # +3 is because of the padding
        elif t_without_id == REC_TYPE.ERROR_INFO:
            #print(f"Error info found at {i}")
            ts, error = struct.unpack('<LL', log_b[i: i + 8])
            if first_ts == -1:
                first_ts = ts
            error_info.append({'ts': ts,
                               'error': error})
            i += 4 + 4
        else:
            print(t)
            print(f"ERROR at {i}")
            break
        last_ts = ts
except Exception as e:
    print(f'Parsing ended with error: {e}')
finally:
    print(f'Parsing ended at position: {i}/{len(log_b)}\nFirst timestamp: {first_ts / 1000}s; Last timestamp: {last_ts / 1000}s\n')

Parsing ended with error: unpack requires a buffer of 12 bytes
Parsing ended at position: 535548/535552
First timestamp: 20.149s; Last timestamp: 46.399s



In [11]:
ev_liftoff_ts = 0
for event in event_info:
    event['event'] = EVENT_MAP[event['event']]
    print(f"{event['ts']} -- {event['event']} -- out_idx: {event['out_idx']}")
    if event['event'] == 'EV_LIFTOFF':
        ev_liftoff_ts = event['ts']

30530 -- EV_MOVING -- out_idx: 255


In [12]:
for state in flight_states:
    state['state'] = STATE_MAP[state['state']]
    print(f"{state['ts']} -- {state['state']}")

30530 -- MOVING


In [13]:
for error in error_info:
    error['error'] = ERROR_MAP[error['error']]
    print(f"{error['ts']} -- {error['error']}")

In [14]:
imu_df = pd.DataFrame(imu)
baro_df = pd.DataFrame(baro)
accelerometer_df = pd.DataFrame(accelerometer)
flight_info_df = pd.DataFrame(flight_info)
orientation_info_df = pd.DataFrame(orientation_info)
filtered_data_info_df = pd.DataFrame(filtered_data_info)
covariance_df = pd.DataFrame(covariance_info)
flight_info_df['lower_bound'] = flight_info_df['height'] + 3 * (covariance_df['height_cov']) ** 0.5
flight_info_df['upper_bound'] = flight_info_df['height'] - 3 * (covariance_df['height_cov']) ** 0.5
sensor_info_df = pd.DataFrame(sensor_info)
event_info_df = pd.DataFrame(event_info)
error_info_df = pd.DataFrame(error_info)
magneto_df = pd.DataFrame(magneto)
flight_states_df = pd.DataFrame(flight_states)

## Save Raw Logs

In [15]:
imu_df.to_csv(f'{raw_output_dir}/{base_name} - imu_raw.csv')
baro_df.to_csv(f'{raw_output_dir}/{base_name} - baro_raw.csv')
accelerometer_df.to_csv(f'{raw_output_dir}/{base_name} - accelerometer_raw.csv')
orientation_info_df.to_csv(f'{raw_output_dir}/{base_name} - orientation_info_raw.csv')
flight_info_df.to_csv(f'{raw_output_dir}/{base_name} - flight_info_raw.csv')
filtered_data_info_df.to_csv(f'{raw_output_dir}/{base_name} - filtered_data_info_raw.csv')
covariance_df.to_csv(f'{raw_output_dir}/{base_name} - covariance_df_raw.csv')
sensor_info_df.to_csv(f'{raw_output_dir}/{base_name} - sensor_info_raw.csv')
event_info_df.to_csv(f'{raw_output_dir}/{base_name} - event_info_raw.csv')
error_info_df.to_csv(f'{raw_output_dir}/{base_name} - error_info_raw.csv')
magneto_df.to_csv(f'{raw_output_dir}/{base_name} - magneto_info_raw.csv')
flight_states_df.to_csv(f'{raw_output_dir}/{base_name} - flight_states_raw.csv')
orientation_info_df

,ts,q0_estimated,q1_estimated,q2_estimated,q3_estimated,q0_raw,q1_raw,q2_raw,q3_raw
0,20150,9331,-484,-645,3503,9331,-484,-645,3503
1,20160,9331,-487,-645,3503,9331,-487,-645,3503
2,20170,9331,-486,-648,3503,9331,-486,-648,3503
3,20180,9331,-488,-647,3502,9331,-488,-647,3502
4,20190,9331,-488,-649,3502,9331,-488,-649,3502
...,...,...,...,...,...,...,...,...,...
2620,46350,9328,-477,-460,3540,9328,-477,-460,3540
2621,46360,9328,-474,-460,3540,9328,-474,-460,3540
2622,46370,9328,-474,-456,3541,9328,-474,-456,3541
2623,46380,9328,-474,-455,3541,9328,-474,-455,3541


## Process Logs

In [16]:
def offset_col(df, col, offset):
    df[col] -= offset


def scale_col(df, col, scale_factor):
    df[col] /= scale_factor

zero_ts = first_ts
if ev_liftoff_ts > 0:
    zero_ts = ev_liftoff_ts
    
offset_col(imu_df, 'ts', zero_ts)
offset_col(baro_df, 'ts', zero_ts)
offset_col(magneto_df, 'ts', zero_ts)
offset_col(accelerometer_df, 'ts', zero_ts)
offset_col(orientation_info_df, 'ts', zero_ts)
offset_col(flight_info_df, 'ts', zero_ts)
offset_col(filtered_data_info_df, 'ts', zero_ts)
offset_col(covariance_df, 'ts', zero_ts)
#offset_col(sensor_info_df, 'ts', zero_ts)

scale_col(imu_df, 'ts', 1000)
scale_col(baro_df, 'ts', 1000)
scale_col(magneto_df, 'ts', 1000)
scale_col(accelerometer_df, 'ts', 1000)
scale_col(orientation_info_df, 'ts', 1000)
scale_col(flight_info_df, 'ts', 1000)
scale_col(filtered_data_info_df, 'ts', 1000)
scale_col(covariance_df, 'ts', 1000)
#scale_col(sensor_info_df, 'ts', 1000)

if len(event_info_df) > 0:
    offset_col(event_info_df, 'ts', zero_ts)
    scale_col(event_info_df, 'ts', 1000)
if len(flight_states_df) > 0:
    offset_col(flight_states_df, 'ts', zero_ts)
    scale_col(flight_states_df, 'ts', 1000)
if len(error_info_df) > 0:
    offset_col(error_info_df, 'ts', zero_ts)
    scale_col(error_info_df, 'ts', 1000)

scale_col(imu_df, 'Gx', 16.4)
scale_col(imu_df, 'Gy', 16.4)
scale_col(imu_df, 'Gz', 16.4)
scale_col(imu_df, 'Ax', 1024)
scale_col(imu_df, 'Ay', 1024)
scale_col(imu_df, 'Az', 1024)

scale_col(orientation_info_df, 'q0_estimated', 1000)
scale_col(orientation_info_df, 'q1_estimated', 1000)
scale_col(orientation_info_df, 'q2_estimated', 1000)
scale_col(orientation_info_df, 'q3_estimated', 1000)
scale_col(orientation_info_df, 'q0_raw', 1000)
scale_col(orientation_info_df, 'q1_raw', 1000)
scale_col(orientation_info_df, 'q2_raw', 1000)
scale_col(orientation_info_df, 'q3_raw', 1000)

scale_col(baro_df, 'T', 100)
imu_df

,ts,id,Gx,Gy,Gz,Ax,Ay,Az
0,0.01,IMU0,0.609756,0.548780,0.182927,-0.147461,0.041992,0.946289
1,0.01,IMU1,-0.182927,0.182927,-0.304878,-0.136719,0.042969,0.908203
2,0.02,IMU0,0.731707,0.609756,0.182927,-0.149414,0.045898,0.950195
3,0.02,IMU1,-0.121951,0.243902,-0.304878,-0.137695,0.047852,0.914062
4,0.03,IMU0,0.548780,0.548780,0.182927,-0.150391,0.041992,0.947266
...,...,...,...,...,...,...,...,...
5245,26.23,IMU1,-0.121951,0.060976,-0.304878,-0.100586,0.054688,0.916016
5246,26.24,IMU0,0.609756,0.609756,0.243902,-0.113281,0.053711,0.953125
5247,26.24,IMU1,-0.182927,0.182927,-0.243902,-0.104492,0.054688,0.914062
5248,26.25,IMU0,0.670732,0.487805,0.243902,-0.115234,0.054688,0.953125


## Save Processed Logs

In [17]:
imu_df.to_csv(f'{processed_output_dir}/{base_name} - imu_processed.csv')
baro_df.to_csv(f'{processed_output_dir}/{base_name} - baro_processed.csv')
accelerometer_df.to_csv(f'{processed_output_dir}/{base_name} - accelerometer_processed.csv')
flight_info_df.to_csv(f'{processed_output_dir}/{base_name} - flight_info_processed.csv')
orientation_info_df.to_csv(f'{processed_output_dir}/{base_name} - orientation_info_processed.csv')
filtered_data_info_df.to_csv(f'{processed_output_dir}/{base_name} - filtered_data_info_processed.csv')
covariance_df.to_csv(f'{processed_output_dir}/{base_name} - covariance_df_processed.csv')
sensor_info_df.to_csv(f'{processed_output_dir}/{base_name} - sensor_info_processed.csv')
event_info_df.to_csv(f'{processed_output_dir}/{base_name} - event_info_processed.csv')
error_info_df.to_csv(f'{processed_output_dir}/{base_name} - error_info_processed.csv')
magneto_df.to_csv(f'{processed_output_dir}/{base_name} - magneto_processed.csv')
flight_states_df.to_csv(f'{processed_output_dir}/{base_name} - flight_states_processed.csv')

## Generate Plots

In [18]:
def add_flight_states_and_events(fig, flight_states_df, event_info_df):
    for _, state in flight_states_df.iterrows():
        fig.add_vline(x=state['ts'], line_width=1, line_dash="dash", line_color="green", annotation_text=state['state'])

    for _, event in event_info_df.iterrows():
        if event['out_idx'] == 0:
            fig.add_vline(x=event['ts'], line_width=1, line_dash="dot", line_color="orange",
                          annotation_text=event['event'], annotation_y=0.03)
            
    for _, error in error_info_df.iterrows():
        fig.add_vline(x=error['ts'], line_width=1, line_dash="dot", line_color="red",
                      annotation_text=error['error'], annotation_y=0.03)    

In [19]:
imu_plots = plot_imu_data(imu_df, flight_states_df, event_info_df, True)
magneto_plots = plot_magneto_data(magneto_df, flight_states_df, event_info_df, True)
baro_plots = plot_baro_data(baro_df, flight_states_df, event_info_df, True)
state_plots = plot_flight_info(flight_info_df, filtered_data_info_df, flight_states_df, event_info_df, False, True)

In [20]:
out_name = log.split('/')[-1].split('.')[0]
figures_to_html(imu_plots, baro_plots, magneto_plots, state_plots, f'{plot_output_dir}/{base_name} - plots.html')

In [21]:
plot_imu_data(imu_df, flight_states_df, event_info_df)

In [22]:
plot_baro_data(baro_df, flight_states_df, event_info_df)

In [23]:
plot_magneto_data(magneto_df, flight_states_df, event_info_df)

In [24]:
plot_flight_info(flight_info_df, filtered_data_info_df, flight_states_df, event_info_df)

In [25]:
sensor_info_df.max()

Series([], dtype: float64)